In [ ]:
#necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

csv_path = '/content/koniq10k_distributions_sets.csv'
images = pd.read_csv(csv_path, usecols=['image_name', 'MOS'])
images.head()


,image_name,MOS
0,10004473376.jpg,77.383621
1,10007357496.jpg,68.728571
2,10007903636.jpg,78.628571
3,10009096245.jpg,77.243750
4,100117038.jpg,75.112500


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split
import tensorflow as tf

image_dir = Path('/content/512x384')

In [ ]:
train_df, test_df = train_test_split(images, train_size=0.8, shuffle=True, random_state=1)
train_df['image_name'] = '/content/512x384/' + train_df['image_name']
test_df['image_name'] = '/content/512x384/' + test_df['image_name']
train_df

,image_name,MOS
8048,/content/512x384/8195268203.jpg,26.334071
6731,/content/512x384/6935202616.jpg,45.565421
4989,/content/512x384/5677087869.jpg,77.066514
6686,/content/512x384/6906948754.jpg,72.548673
2684,/content/512x384/3973001148.jpg,38.159664
...,...,...
2895,/content/512x384/4169643886.jpg,48.911111
7813,/content/512x384/8045790915.jpg,55.006303
905,/content/512x384/2059981058.jpg,58.750000
5192,/content/512x384/5831302222.jpg,43.718220


In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.25

)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_name',
    y_col='MOS',
    #target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_name',
    y_col='MOS',
    #target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_name',
    y_col='MOS',
    #target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

Found 6044 validated image filenames.
Found 2014 validated image filenames.
Found 2015 validated image filenames.


In [ ]:
from tensorflow.keras.layers import Layer, Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Multiply, Add
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

class CBAM(Layer):
    def __init__(self, ratio=8, **kwargs):
        super(CBAM, self).__init__(**kwargs)
        self.ratio = ratio

    def build(self, input_shape):
        self.channels = input_shape[-1]
        self.avg_pool = GlobalAveragePooling2D()
        self.max_pool = GlobalMaxPooling2D()

        self.shared_dense1 = Dense(self.channels // self.ratio, activation='relu', kernel_initializer='he_normal', use_bias=True)
        self.shared_dense2 = Dense(self.channels, kernel_initializer='he_normal', use_bias=True)

        super(CBAM, self).build(input_shape)

    def call(self, inputs):
        avg_pool = self.avg_pool(inputs)
        max_pool = self.max_pool(inputs)

        avg_dense = self.shared_dense2(self.shared_dense1(avg_pool))
        max_dense = self.shared_dense2(self.shared_dense1(max_pool))

        attention = self.add([avg_dense, max_dense])
        attention = K.sigmoid(attention)

        attention = Reshape((1, 1, self.channels))(attention)
        return Multiply()([inputs, attention])

    def compute_output_shape(self, input_shape):
        return input_shape

In [ ]:
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D, Dense, Reshape, Multiply, Add, Concatenate, Activation, Conv2D

def channel_attention_module(x, ratio=8):
    batch, _, _, channel = x.shape

    ## Shared layers
    l1 = Dense(channel//ratio, activation="relu", use_bias=False)
    l2 = Dense(channel, use_bias=False)

    ## Global Average Pooling
    x1 = GlobalAveragePooling2D()(x)
    x1 = l1(x1)
    x1 = l2(x1)

    ## Global Max Pooling
    x2 = GlobalMaxPooling2D()(x)
    x2 = l1(x2)
    x2 = l2(x2)

    ## Add both the features and pass through sigmoid
    feats = x1 + x2
    feats = Activation("sigmoid")(feats)
    feats = Multiply()([x, feats])

    return feats

def spatial_attention_module(x):
    ## Average Pooling
    x1 = tf.reduce_mean(x, axis=-1)
    x1 = tf.expand_dims(x1, axis=-1)

    ## Max Pooling
    x2 = tf.reduce_max(x, axis=-1)
    x2 = tf.expand_dims(x2, axis=-1)

    ## Concatenat both the features
    feats = Concatenate()([x1, x2])
    ## Conv layer
    feats = Conv2D(1, kernel_size=7, padding="same", activation="sigmoid")(feats)
    feats = Multiply()([x, feats])

    return feats

def cbam(x):
    x = channel_attention_module(x)
    x = spatial_attention_module(x)
    return x

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D, Dense, Reshape, Multiply, Add

from tensorflow.keras.applications import InceptionResNetV2, EfficientNetB0
input_shape = (224, 224, 3)

backbone = InceptionResNetV2(input_shape=input_shape, weights='imagenet', include_top=False)
model = Sequential()
model.add(backbone)

# Adding custom layers
model.add(Dropout(0.2))
model.add(CBAMLayer()(model.output))
model.add(GlobalAveragePooling2D())


# Final layer for regression (one neuron)
model.add(Dense(1, activation='linear'))

optimizer = Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=[plcc])  # Assuming 'plcc' is a defined metric
print(model.summary())


history = model.fit(
    train_images,
    validation_data=val_images,
    batch_size=32,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True
        )
    ]
)

In [ ]:
from tensorflow.keras.models import Model

from tensorflow.keras.applications import InceptionResNetV2
input_shape = (512, 384, 3)

backbone = InceptionResNetV2(input_shape=input_shape, weights='imagenet', include_top=False)
model = Sequential()
model.add(backbone)


model.add(GlobalAveragePooling2D())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Final layer for regression (one neuron)
model.add(Dense(1, activation='linear'))

optimizer = Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=[plcc])  # Assuming 'plcc' is a defined metric
print(model.summary())


history = model.fit(
    train_images,
    validation_data=val_images,
    batch_size=32,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True
        )    ]
)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Funct  (None, 14, 10, 1536)      54336736  
 ional)                                                          
                                                                 
 global_average_pooling2d_2  (None, 1536)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_8 (Dense)             (None, 2048)              3147776   
                                                                 
 dropout_6 (Dropout)         (None, 2048)              0         
                                                                 
 dense_9 (Dense)             (None, 1024)              2098176   
                                                                 
 dropout_7 (Dropout)         (None, 1024)             

In [ ]:
def plcc(y_true, y_pred):
    y_true = tf.squeeze(y_true)  # Ensure shapes are compatible
    y_pred = tf.squeeze(y_pred)
    mean_true = tf.reduce_mean(y_true)
    mean_pred = tf.reduce_mean(y_pred)

    centered_true = y_true - mean_true
    centered_pred = y_pred - mean_pred

    numerator = tf.reduce_sum(centered_true * centered_pred)
    denominator = tf.sqrt(tf.reduce_sum(tf.square(centered_true)) * tf.reduce_sum(tf.square(centered_pred)))

    plcc_value = numerator / denominator

    return plcc_value

In [ ]:
predicted_mos = model.predict(test_images)  # Assuming this contains your model's predicted values
true_mos = test_images.labels  # Assuming this contains the true labels
true_mos = true_mos.reshape((-1, 1))
true_mos = true_mos.astype(np.float32)
plcc_value = plcc(true_mos, predicted_mos)
print("Test PLCC: {:.5f}".format(plcc_value))

63/63 [==============================] - 19s 257ms/step
Test PLCC: 0.83712


In [ ]:
!unzip /content/drive/MyDrive/koniq10k_512x384.zip

Streaming output truncated to the last 5000 lines.
  inflating: 512x384/9557331185.jpg  
  inflating: 512x384/8036386438.jpg  
  inflating: 512x384/18238731.jpg    
  inflating: 512x384/8148747862.jpg  
  inflating: 512x384/9442161547.jpg  
  inflating: 512x384/3117619556.jpg  
  inflating: 512x384/5489747456.jpg  
  inflating: 512x384/5764115414.jpg  
  inflating: 512x384/3272637437.jpg  
  inflating: 512x384/7035272289.jpg  
  inflating: 512x384/7032481793.jpg  
  inflating: 512x384/3951898034.jpg  
  inflating: 512x384/6411944553.jpg  
  inflating: 512x384/564783596.jpg   
  inflating: 512x384/2140758242.jpg  
  inflating: 512x384/8452461056.jpg  
  inflating: 512x384/5305438208.jpg  
  inflating: 512x384/6239374380.jpg  
  inflating: 512x384/4258446622.jpg  
  inflating: 512x384/4259335567.jpg  
  inflating: 512x384/4940996206.jpg  
  inflating: 512x384/302681636.jpg   
  inflating: 512x384/5732196042.jpg  
  inflating: 512x384/9234807959.jpg  
  inflating: 512x384/5298734322.jpg  